# MODEL TRAINING 
## using 5 cross validation

In [1]:
import joblib
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [2]:
tr_feature = pd.read_csv('train_features.csv')
tr_label = pd.read_csv('train_label.csv')
vl_feature = pd.read_csv('val_features.csv')
vl_label = pd.read_csv('val_label.csv')
ts_feature = pd.read_csv('test_features.csv')
ts_label = pd.read_csv('test_label.csv')

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [4]:
tr_feature = scaler.fit_transform(tr_feature)
vl_feature = scaler.fit_transform(vl_feature)
ts_feature = scaler.fit_transform(ts_feature)

In [5]:
def print_result(results):
    print(f'BEST PARAMS :{results.best_params_}')
    mean = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    
    for mean,std,param in zip(mean,stds,params):
        print(f'{round(mean,3)} +/-{round(std,3)} for{param}')

check the last commit in which multiple parameters are used
now i'am working on the best parameter c= 30 and kernel = 'linear'
in my older commit the model take more time to fit the model

In [6]:

svm_model = SVC()
parameters = {
    'C' :[30],
    'kernel' :['linear','rbf']
}
cv = GridSearchCV(svm_model,parameters,cv=5)
cv.fit(tr_feature,tr_label.values.ravel())


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [30], 'kernel': ['linear', 'rbf']})

In [7]:
print_result(cv)

BEST PARAMS :{'C': 30, 'kernel': 'rbf'}
0.78 +/-0.0 for{'C': 30, 'kernel': 'linear'}
0.828 +/-0.005 for{'C': 30, 'kernel': 'rbf'}


In [8]:
joblib.dump(cv.best_estimator_,'SVM_MDL.pkl')

['SVM_MDL.pkl']

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from time import time

In [10]:
model = {'SVM_MDL.pkl':joblib.load('SVM_MDL.pkl')}

In [11]:
def evaluuate_model(name,model,features,labels):
    start = time()
    pred = model.predict(features)
    end = time()
    accuracy = round(accuracy_score(labels,pred),3)
    precision = round(precision_score(labels,pred),3)
    recall = round(recall_score(labels,pred),3)
    print(f'{name}--Accuracy: {accuracy}/ Precision: {precision} / Recall: {recall} / Latency: {round((end-start)*1000,1)}ms')

### Geting accuracy for validation data

In [12]:
for name,mdl in model.items():
    evaluuate_model(name,mdl,vl_feature,vl_label)

SVM_MDL.pkl--Accuracy: 0.812/ Precision: 0.619 / Recall: 0.364 / Latency: 141.0ms


### Geting accuracy for Test data

In [13]:
evaluuate_model('SVM',model['SVM_MDL.pkl'],ts_feature,ts_label)

SVM--Accuracy: 0.837/ Precision: 0.649 / Recall: 0.435 / Latency: 136.5ms


In [ ]:
SVM--Accuracy: 0.837/ Precision: 0.649 / Recall: 0.435 / Latency: 136.5ms